In [1]:
from cartopy.io import shapereader
import shapefile
from json import dumps

import folium
from folium import plugins
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
%matplotlib inline

In [2]:
reader = shapefile.Reader('zip_data/plz-gebiete.shp')
fields = reader.fields[1:]
field_names = [field[0] for field in fields]

data_geo = []
data = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    if atr['plz'][0:2] == '80':
        cord = geom['coordinates']
        temp = []
        for i in range (0,len(cord[0]),1):
            temp.append(cord[0][i])
        loc = np.array(temp).mean(axis=0)
        data.append(dict(plz=atr['plz'],longitude = loc[0], latitude= loc[1] ))
        geom['coordinates'] = tuple(temp),
        data_geo.append(dict(type="Feature", geometry=geom, properties=atr))

In [3]:
def shape2json(fname, outfile="ZIP.json", res=1, ZIP_Codes = []):
    reader = shapefile.Reader(fname)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]

    data = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        cord = geom['coordinates']
        temp = []
        for i in range (0,len(cord[0]),res):
            temp.append(cord[0][i])
        geom['coordinates'] = tuple(temp),
        if atr['plz'] in ZIP_Codes:
            data.append(dict(type="Feature", geometry=geom, properties=atr))

    with open(outfile, "w") as geojson:
        geojson.write(dumps({"type": "FeatureCollection",
                             "features": data}, indent=2) + "\n")

In [4]:
ZIP_i2s = lambda x: str(x) if len(str(x)) > 4 else '0'+str(x)
geo_cols = ['Country', 'ZIP', 'City', 'State', 'ABB', 'N1', 'N2', 'Region', 'N3', 'Latitude', 'Longitude', 'Acc']
geocode_df = pd.read_csv('zip_data/DE_ZIP.csv', sep = ';', header=None, names=geo_cols)

population_df = pd.read_csv('zip_data/plz_einwohner.csv')


geocode_df.drop(['N1','N2','N3'],inplace=True, axis=1)
geocode_df.drop_duplicates(subset=['ZIP'], keep='first', inplace=True)
geocode_df['ZIP'] = geocode_df['ZIP'].apply(ZIP_i2s)
population_df['ZIP'] = population_df['plz'].apply(ZIP_i2s)
population_df.drop('plz', inplace=True, axis=1)
info_ZIP_df = pd.merge(population_df,geocode_df,how='left',left_on='ZIP',right_on='ZIP',sort=False)
info_ZIP_df = info_ZIP_df[~info_ZIP_df['Longitude'].isna()]
info_ZIP_df = info_ZIP_df[info_ZIP_df['einwohner']>0]

In [5]:
geocode_df.head()

,Country,ZIP,City,State,ABB,Region,Latitude,Longitude,Acc
0,Column1,Column2,Column3,Column4,Column5,Column8,Column10,Column11,Column12
1,DE,01067,Dresden Friedrichstadt,Sachsen,SN,Kreisfreie Stadt Dresden,510587,137172,4
4,DE,01069,Dresden,Sachsen,SN,Kreisfreie Stadt Dresden,51043,137373,4
5,DE,01097,Dresden,Sachsen,SN,Kreisfreie Stadt Dresden,510714,137399,4
6,DE,01099,Dresden,Sachsen,SN,Kreisfreie Stadt Dresden,510783,138051,4


In [6]:
new_df = geocode_df[geocode_df['City']== 'München'][['ZIP']].reset_index(drop=True)
new_df['Count'] = np.random.randint(0,10,(74))
new_df.rename(columns={'ZIP': 'plz'},inplace=True)
new_df = new_df.tail(71)

In [7]:
new_df.head()

,plz,Count
3,80336,7
4,80337,3
5,80339,2
6,80469,3
7,80538,7


In [8]:
ZIP_Mun = geocode_df[geocode_df['City']== 'München']['ZIP'].values
shape2json('zip_data/plz-gebiete.shp', outfile="ZIP_MUN.json", res=1, ZIP_Codes = ZIP_Mun)

In [9]:
mapa = folium.Map(location=[48.1452, 11.5668], zoom_start=10, )
g = folium.Choropleth(geo_data='ZIP_MUN.json', data= new_df, columns=['plz', 'Count'], fill_color='Reds', \
                  line_color='blue',line_weight=2, key_on='feature.properties.plz',)
g.popup_function = lambda feature: 1
g.add_to(mapa)
mapa

In [12]:
#info_ZIP_df[info_ZIP_df['ZIP'].str[0:2] == '80']

In [16]:
#from folium.features import DivIcon

In [15]:
#text = '80333'
#mapb = folium.Map(location=[48.1452, 11.5668], zoom_start=10, ) 
#folium.Marker(loc[0], icon=DivIcon(       html='<div style="font-size: 10pt">%s</div>' % text,)).add_to(mapb)
#mapb